# [E-14] Movielens 영화 추천

이번 프로젝트는 추천시스템의 기본적인 원리와 구성을 파악하고 MovieLens 데이터를 통해 간단하고 효과적인 영화 추천시스템을 구현해보는 것을 목표로 한다.

<hr>

## 1. 데이터 준비 및 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남김
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿈
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# 데이터프레임 합치기
data = pd.merge(ratings, movies, how = 'left', on = 'movie_id')
data.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [6]:
# 사용하지 않는 컬럼 제거
data = data.drop(columns=['timestamp', 'genre', 'movie_id'])
data.head()

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"


<hr>

## 2. 데이터 확인

In [7]:
print('유니크한 영화 개수는 {}개 입니다.'.format(data['title'].nunique()))
print('유니크한 사용자 수는 {}개 입니다.'.format(data['user_id'].nunique()))

유니크한 영화 개수는 3628개 입니다.
유니크한 사용자 수는 6039개 입니다.


In [8]:
# 인기 많은 영화 30편
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [9]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [10]:
# 유저별 counts 중앙값에 대한 통계
user_median = data.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

<hr>

## 3. 내가 선호하는 영화 추가

In [11]:
#내가 좋아하는 영화 5편 리스트
my_favorite = ['Titanic (1997)', 'Saving Private Ryan (1998)', 'Shawshank Redemption, The (1994)', 
               'Matrix, The (1999)', 'Forrest Gump (1994)']

#'bongkim'이라는 user_id가 위 영화들의 평점을 5점으로 줬다고 가정
my_rating = pd.DataFrame({'user_id' : ['bongkim']*5, 'title' : my_favorite, 'counts' : [5]*5})

#user_id에 'bongkim'가 없으면 my_favorite 데이터 추가
if not data.isin({'user_id' : ['bongkim']})['user_id'].any() :
    data = data.append(my_rating, ignore_index = True)
    
data.tail(10)

,user_id,counts,title
836473,6040,3,Platoon (1986)
836474,6040,5,"Crying Game, The (1992)"
836475,6040,5,Welcome to the Dollhouse (1995)
836476,6040,4,Sophie's Choice (1982)
836477,6040,4,E.T. the Extra-Terrestrial (1982)
836478,bongkim,5,Titanic (1997)
836479,bongkim,5,Saving Private Ryan (1998)
836480,bongkim,5,"Shawshank Redemption, The (1994)"
836481,bongkim,5,"Matrix, The (1999)"
836482,bongkim,5,Forrest Gump (1994)


<hr>

## 4. 인덱싱

In [12]:
# user_id와 title 인덱싱
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

user_idx = {v:k for k,v in enumerate(user_unique)}
title_idx = {v:k for k,v in enumerate(movie_unique)}

print(user_idx['bongkim'])
print(title_idx['Forrest Gump (1994)'])

6039
160


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

#user_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series 구하기 
#혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 
temp_user_data = data['user_id'].map(user_idx.get).dropna()

if len(temp_user_data) == len(data):   #모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   #movie_data['user_id']을 인덱싱 된 Series로 교체. 
else:
    print('user_id column indexing Fail!!')

#temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱. 
temp_title_data = data['title'].map(title_idx.get).dropna()

if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
836478,6039,5,27
836479,6039,5,48
836480,6039,5,157
836481,6039,5,124


<hr>

## 5. CSR matrix 생성

In [14]:
from scipy.sparse import csr_matrix

user_num = data['user_id'].nunique()
movie_num = data['title'].nunique()

csr_data = csr_matrix((data['counts'], (data.user_id, data.title)), shape=(user_num, movie_num))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

<hr>

## 6. 모델링

In [15]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

#Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(
    factors=100, 
    regularization=0.01, 
    use_gpu=False, 
    iterations=15, 
    dtype=np.float32)

#als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose 해 줌
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [16]:
#모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

<hr>

## 7. 예측 결과 확인

In [17]:
#훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
bongkim, titanic = user_idx['bongkim'], title_idx['Titanic (1997)']
bongkim_vector, titanic_vector = als_model.user_factors[bongkim], als_model.item_factors[titanic]

In [18]:
# 내가 좋아하는 영화에 대한 내적
np.dot(bongkim_vector, titanic_vector)

0.5973799

In [19]:
# 관련 없는 영화에 대한 내적
matrix = title_idx['Toy Story (1995)']
matrix_vector = als_model.item_factors[matrix]
np.dot(bongkim_vector, matrix_vector)

0.22206399

<hr>

## 8. 영화 추천

In [20]:
idx_title = {v:k for k,v in title_idx.items()}

def similar_movie(title_name: str):
    title_id = title_idx[title_name]
    similar_title = als_model.similar_items(title_id, N = 15)
    similar_title = [idx_title[i[0]] for i in similar_title]
    return similar_title

In [21]:
#Titanic과 유사도가 높은 영화
similar_movie('Titanic (1997)')

['Titanic (1997)',
 'Jerry Maguire (1996)',
 'City of Angels (1998)',
 'English Patient, The (1996)',
 'Ever After: A Cinderella Story (1998)',
 'Bridges of Madison County, The (1995)',
 "You've Got Mail (1998)",
 "Mr. Holland's Opus (1995)",
 'Ghost (1990)',
 'As Good As It Gets (1997)',
 'Few Good Men, A (1992)',
 'Scent of a Woman (1992)',
 'Message in a Bottle (1999)',
 'Far and Away (1992)',
 'Piano, The (1993)']

In [22]:
user = user_idx['bongkim']

#recommend에서는 user*item CSR Matrix를 받음
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(23, 0.5720055),
 (87, 0.5508232),
 (121, 0.54909277),
 (110, 0.36413705),
 (248, 0.33174655),
 (38, 0.3251287),
 (92, 0.32208765),
 (141, 0.31835815),
 (22, 0.30468222),
 (51, 0.30114335),
 (107, 0.27718538),
 (99, 0.26656103),
 (384, 0.25944316),
 (81, 0.24831334),
 (120, 0.24075498),
 (175, 0.23746929),
 (222, 0.23477192),
 (117, 0.22645807),
 (64, 0.22443539),
 (154, 0.22330378)]

In [23]:
#title로 인덱싱
[idx_title[i[0]] for i in movie_recommended]

["Schindler's List (1993)",
 'Braveheart (1995)',
 'Silence of the Lambs, The (1991)',
 'Groundhog Day (1993)',
 'Good Will Hunting (1997)',
 'Sixth Sense, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Fugitive, The (1993)',
 'Back to the Future (1985)',
 'Fargo (1996)',
 'Jurassic Park (1993)',
 'American Beauty (1999)',
 'Jerry Maguire (1996)',
 'Green Mile, The (1999)',
 'Raiders of the Lost Ark (1981)',
 'Men in Black (1997)',
 'Pulp Fiction (1994)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'As Good As It Gets (1997)']

In [24]:
#기록을 남긴 데이터 중 영화 추천에 기여한 정도
recommended = title_idx["Schindler's List (1993)"]
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_title[i[0]], i[1]) for i in explain[1]]

[('Shawshank Redemption, The (1994)', 0.257231729535086),
 ('Saving Private Ryan (1998)', 0.2121571329694857),
 ('Forrest Gump (1994)', 0.10469705670875648),
 ('Matrix, The (1999)', 4.711157958660545e-05),
 ('Titanic (1997)', -0.010766762960008154)]

<hr>

## 프로젝트를 마치며

예전부터 궁금했던 추천시스템을 직접 만들어 볼 수 있는 프로젝트였다. 이전에 노드에서 추천시스템의 개념에 대한 이해를 했었다면 이번 프로젝트에서는 직접 구현해보면서 예측된 결과를 확인해볼 수 있었다. 데이터는 Movielens를 활용하였다. 먼저 데이터 전처리 과정을 거친 후 CSR matrix를 만들어주고 Implicit AlternatingLeastSquares 모델을 통해 훈련해주었다. 내가 좋아할만한 영화를 추천받는 것이 목표이기 때문에 user_id에 id를 새로 추가해주었고 좋아하는 영화 5개를 골라 평점을 5점으로 하여 새로운 id에 추가해주었다. user_id와 영화에 대한 벡터를 생성해주어 두 벡터에 대한 내적을 구해주었다. 여기서 구한 내적은 얼마나 유사한지에 대한 지표로 사용되었다. 

내가 좋아하는 영화는 'Titanic (1997)', 'Saving Private Ryan (1998)', 'Shawshank Redemption, The (1994)', 'Matrix, The (1999)', 'Forrest Gump (1994)'였다. 그 중 첫번 째인 타이타닉 영화의 벡터와의 내적을 구한 결과, 0.5973799의 값이 나왔다. 관련 없는 영화인 '토이스토리'와의 내적 값이 0.22206399인 것과 비교하면 확실히 유사도가 높다고 판단할 수 있다. 최종적으로 결과물을 확인해본 결과, 유사도가 0.5720055, 0.5508232, 0.54909277 등으로 높은 순의 영화부터 추천 리스트가 만들어졌다. 인덱싱을 통해 각각의 영화 제목을 확인해 본 결과, 'Schindler's List', 'Braveheart', 'Silence of the Lambs, The'였다. 그 중 가장 유사도가 높은 'Schindler's List'에 대한 기여도를 확인해보니 'Shawshank Redemption'의 기여도가 약 25.7%, 'Saving Private Ryan'의 기여도가 약 21.2%였다. 

궁금해서 'Schindler's List'의 줄거리를 찾아보니 전쟁을 배경으로 하고 있고 유대인들을 안전한 곳으로 피신시키는 주인공에 대한 내용이었다. 기여도가 높았던 두 영화 '쇼생크 탈출'과 '라이언 일병 구하기'와 나름 연관성이 있는 내용이었다. 짧은 방학도 주어졌으니 내가 만든 모델이 추천해준 영화를 보는 것도 재밌는 경험이 될 것 같다.